In [162]:
!pip install gunicorn

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import pydicom
from matplotlib import pyplot as plt
import png
import pydicom
import matplotlib.pyplot as plt
import scipy.misc
from skimage import io, transform, feature
from PIL import Image
import os
import re
import numpy as np

## Создаем корректные файлы с путями к файлам 

### Просмотр описания снимков

In [24]:
data.head()

,patient_id,breast density,left or right breast,image view,abnormality id,abnormality type,calc type,calc distribution,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path
0,P_00038,2,LEFT,CC,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,Calc-Test_P_00038_LEFT_CC/1.3.6.1.4.1.9590.100...,Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.9590.1...
1,P_00038,2,LEFT,MLO,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,Calc-Test_P_00038_LEFT_MLO/1.3.6.1.4.1.9590.10...,Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9590....,Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9590....
2,P_00038,2,RIGHT,CC,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00038_RIGHT_CC/1.3.6.1.4.1.9590.10...,Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9590....,Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9590....
3,P_00038,2,RIGHT,CC,2,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00038_RIGHT_CC/1.3.6.1.4.1.9590.10...,Calc-Test_P_00038_RIGHT_CC_2/1.3.6.1.4.1.9590....,Calc-Test_P_00038_RIGHT_CC_2/1.3.6.1.4.1.9590....
4,P_00038,2,RIGHT,MLO,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00038_RIGHT_MLO/1.3.6.1.4.1.9590.1...,Calc-Test_P_00038_RIGHT_MLO_1/1.3.6.1.4.1.9590...,Calc-Test_P_00038_RIGHT_MLO_1/1.3.6.1.4.1.9590...


### Изменяем пути к файлам с кривых на верные

In [27]:
type_dir = ['F:\\CBIS-DDSM\\Calc-Test_', 'F:\\CBIS-DDSM\\Calc-Training_',  'F:\\CBIS-DDSM\\Mass-Test_', 'F:\\CBIS-DDSM\\Mass-Training_']
data_csv = ['calc_case_description_test_set.csv', 'calc_case_description_train_set.csv', 'mass_case_description_test_set.csv', 'mass_case_description_train_set.csv']

In [48]:
for sets in range(4):
    data = pd.read_csv(data_csv[sets])
    path_dir=type_dir[sets]
    for i in range(len(data)):
        path=str(path_dir)+str(data['patient_id'][i])+str('_')+str(data['left or right breast'][i])+str('_')+str(data['image view'][i])
        k=0
        for root, dirs, files in os.walk(path, topdown=True):
            k+=1
            if k==3:
                path_file=str(root)+str('\\')+str(files[0])
                file = path_file.replace('\\', '/')
                data.iloc[i, 11] = str(file[13:])
        if i==len(data)-1:
            print('Сохранено {} изображений из {}'.format(i+1, len(data)))
    data.to_csv('new_'+str(data_csv[sets]))

Сохранено 326 изображений из 326
Сохранено 1546 изображений из 1546
Сохранено 378 изображений из 378
Сохранено 1318 изображений из 1318


## Склеиваем calc и mass датасеты с новыми путями к файлам

In [49]:
new_data_csv = ['new_calc_case_description_test_set.csv', 
                'new_calc_case_description_train_set.csv', 
                'new_mass_case_description_test_set.csv', 
                'new_mass_case_description_train_set.csv']

In [51]:
data_test_1 = pd.read_csv(new_data_csv[0])
data_test_2 = pd.read_csv(new_data_csv[2])

data_test = pd.concat([data_test_1, data_test_2]).reset_index().drop(columns=['Unnamed: 0', 'index'])

In [57]:
data_test.to_csv('test.csv')

In [58]:
data_train_1 = pd.read_csv(new_data_csv[0])
data_train_2 = pd.read_csv(new_data_csv[2])

data_train = pd.concat([data_train_1, data_train_2]).reset_index().drop(columns=['Unnamed: 0', 'index'])

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [59]:
data_train.to_csv('train.csv')

In [60]:
data_train.head()

,ROI mask file path,abnormality id,abnormality type,assessment,breast density,breast_density,calc distribution,calc type,cropped image file path,image file path,image view,left or right breast,mass margins,mass shape,pathology,patient_id,subtlety
0,Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.9590.1...,1,calcification,4,2.0,NaN,CLUSTERED,PUNCTATE-PLEOMORPHIC,Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Calc-Test_P_00038_LEFT_CC/08-29-2017-DDSM-9600...,CC,LEFT,NaN,NaN,BENIGN,P_00038,2
1,Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9590....,1,calcification,4,2.0,NaN,CLUSTERED,PUNCTATE-PLEOMORPHIC,Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9590....,Calc-Test_P_00038_LEFT_MLO/08-29-2017-DDSM-176...,MLO,LEFT,NaN,NaN,BENIGN,P_00038,2
2,Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9590....,1,calcification,2,2.0,NaN,NaN,VASCULAR,Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9590....,Calc-Test_P_00038_RIGHT_CC/08-29-2017-DDSM-284...,CC,RIGHT,NaN,NaN,BENIGN_WITHOUT_CALLBACK,P_00038,5
3,Calc-Test_P_00038_RIGHT_CC_2/1.3.6.1.4.1.9590....,2,calcification,2,2.0,NaN,NaN,VASCULAR,Calc-Test_P_00038_RIGHT_CC_2/1.3.6.1.4.1.9590....,Calc-Test_P_00038_RIGHT_CC/08-29-2017-DDSM-284...,CC,RIGHT,NaN,NaN,BENIGN_WITHOUT_CALLBACK,P_00038,5
4,Calc-Test_P_00038_RIGHT_MLO_1/1.3.6.1.4.1.9590...,1,calcification,2,2.0,NaN,NaN,VASCULAR,Calc-Test_P_00038_RIGHT_MLO_1/1.3.6.1.4.1.9590...,Calc-Test_P_00038_RIGHT_MLO/08-29-2017-DDSM-85...,MLO,RIGHT,NaN,NaN,BENIGN_WITHOUT_CALLBACK,P_00038,5
